In [1]:
conda --version

conda 4.8.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pymoo

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pymoo

In [4]:
conda create -n pymoo -y python==3.7 numpy

Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\George\anaconda3\envs\pymoo

  added / updated specs:
    - numpy
    - python==3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vs2015_runtime-14.16.27012 |       hf0eaf9b_3         1.2 MB
    ------------------------------------------------------------
                                           Total:         1.2 MB

The following NEW packages will be INSTALLED:

  blas               pkgs/main/win-64::blas-1.0-mkl
  certifi            pkgs/main/win-64::certifi-2020.6.20-py37_0
  icc_rt             pkgs/main/win-64::icc_rt-2019.0.0-h0cc432a_1
  intel-openmp       pkgs/main/win-64::intel-openmp-2020.1-216
  mkl                pkgs/main/win-64::mkl-2020.1-216
  mkl-servic

In [5]:
conda update -n base -c defaults conda

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
conda activate pymoo


Note: you may need to restart the kernel to use updated packages.


python -c "from pymoo.util.function_loader import is_compiled;print('Compiled Extensions: ', is_compiled())"

In [7]:
make compile

SyntaxError: invalid syntax (<ipython-input-7-3a56e0662134>, line 1)

# Algorithms
## NSGA2

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

problem = get_problem("zdt1")

algorithm = NSGA2(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 200),
               seed=1,
               verbose=False)

plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, color="red")
plot.show()

## ZDT1

In [ ]:
from pymoo.factory import get_problem

zdt1 = get_problem("zdt1")

# Example Problem
### Define Problem

In [ ]:
import numpy as np
from pymoo.util.misc import stack
from pymoo.model.problem import Problem

class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = x[:,0]**2 + x[:,1]**2
        f2 = (x[:,0]-1)**2 + x[:,1]**2

        g1 = 2*(x[:, 0]-0.1) * (x[:, 0]-0.9) / 0.18
        g2 = - 20*(x[:, 0]-0.4) * (x[:, 0]-0.6) / 4.8

        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([g1, g2])


    # --------------------------------------------------
    # Pareto-front - not necessary but used for plotting
    # --------------------------------------------------
    def _calc_pareto_front(self, flatten=True, **kwargs):
        f1_a = np.linspace(0.1**2, 0.4**2, 100)
        f2_a = (np.sqrt(f1_a) - 1)**2

        f1_b = np.linspace(0.6**2, 0.9**2, 100)
        f2_b = (np.sqrt(f1_b) - 1)**2

        a, b = np.column_stack([f1_a, f2_a]), np.column_stack([f1_b, f2_b])
        return stack(a, b, flatten=flatten)

    # --------------------------------------------------
    # Pareto-set - not necessary but used for plotting
    # --------------------------------------------------
    def _calc_pareto_set(self, flatten=True, **kwargs):
        x1_a = np.linspace(0.1, 0.4, 50)
        x1_b = np.linspace(0.6, 0.9, 50)
        x2 = np.zeros(50)

        a, b = np.column_stack([x1_a, x2]), np.column_stack([x1_b, x2])
        return stack(a,b, flatten=flatten)

problem = MyProblem()

### Initialize Algorithm

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation

algorithm = NSGA2(
    pop_size=40,
    n_offsprings=10,
    sampling=get_sampling("real_random"),
    crossover=get_crossover("real_sbx", prob=0.9, eta=15),
    mutation=get_mutation("real_pm", eta=20),
    eliminate_duplicates=True
)

### Define a Termination Criterion
Instead of number of generations (or iterations) other criteria such as the number of function evaluations or the improvement in design or objective space from the last to the current generation can be used.

In [ ]:
from pymoo.factory import get_termination

termination = get_termination("n_gen", 40)

### Solve Problem

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               pf=problem.pareto_front(use_cache=False),
               save_history=True,
               verbose=False)

### Visualize

In [ ]:
from pymoo.visualization.scatter import Scatter

# get the pareto-set and pareto-front for plotting
ps = problem.pareto_set(use_cache=False, flatten=False)
pf = problem.pareto_front(use_cache=False, flatten=False)

# Design Space
plot = Scatter(title = "Design Space", axis_labels="x")
plot.add(res.X, s=30, facecolors='none', edgecolors='r')
plot.add(ps, plot_type="line", color="black", alpha=0.7)
plot.do()
plot.apply(lambda ax: ax.set_xlim(-0.5, 1.5))
plot.apply(lambda ax: ax.set_ylim(-2, 2))
plot.show()

# Objective Space
plot = Scatter(title = "Objective Space")
plot.add(res.F)
plot.add(pf, plot_type="line", color="black", alpha=0.7)
plot.show()

## Performance Tracking

In [ ]:
import matplotlib.pyplot as plt
from pymoo.performance_indicator.hv import Hypervolume

# create the performance indicator object with reference point (4,4)
metric = Hypervolume(ref_point=np.array([1.0, 1.0]))

# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]

# calculate for each generation the HV metric
hv = [metric.calc(f) for f in obj_and_feasible_each_gen]

# function evaluations at each snapshot
n_evals = np.array([a.evaluator.n_eval for a in res.history])

# visualze the convergence curve
plt.plot(n_evals, hv, '-o')
plt.title("Convergence")
plt.xlabel("Function Evaluations")
plt.ylabel("Hypervolume")
plt.show()

# Interface
## Interface: Minimize 

In [ ]:
def minimize(problem,
             algorithm,
             termination=None,
             seed=None,
             verbose=False,
             display=None,
             callback=None,
             return_least_infeasible=False,
             save_history=False
             )

seed: Most algorithms underly some randomness. Setting the seed to a positive integer value ensure reproducible results. If not provided a random seed will be set automatically and the used integer will be stored in the Result object.

return_least_infeasible: Whether if the algorithm can not find a feasible solution the least infeasible solution should be returned. By default False.

## Interface: Termination Criteria

In [ ]:
from pymoo.util.termination.default import MultiObjectiveDefaultTermination

termination = MultiObjectiveDefaultTermination(
    x_tol=1e-8,
    cv_tol=1e-6,
    f_tol=0.0025,
    nth_gen=5,
    n_last=30,
    n_max_gen=1000,
    n_max_evals=100000
)

tol: What is the tolerance in the objective space in average. If the value is below this bound we terminate.

n_last: To make the criterion more robust, we consider the last n generations and take the maximum. This considers the worst case in a window.

n_max_gen: As a fallback the generation number can be used. For some problems the termination criterion might not be reached, however, an upper bound for generations can be defined to stop in that case.

nth_gen: Defines whenever the termination criterion is calculated. By default, every 10th generation.

## Interface: Callback

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_problem
from pymoo.model.callback import Callback
from pymoo.optimize import minimize


class MyCallback(Callback):

    def __init__(self) -> None:
        super().__init__()
        self.data["best"] = []

    def notify(self, algorithm):
        self.data["best"].append(algorithm.pop.get("F").min())


problem = get_problem("sphere")

algorithm = GA(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 20),
               seed=1,
               callback=MyCallback(),
               save_history=True,
               verbose=True)

val = res.algorithm.callback.data["best"]
plt.plot(np.arange(len(val)), val)
plt.show()

By using the history object the same as above can be achieved by using the stored information during the run:

In [ ]:
val = [e.pop.get("F").min() for e in res.history]
plt.plot(np.arange(len(val)), val)
plt.show()

## Interface: Display
To change what is displayed for each gen pass display object to minimize function. 

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.util.display import Display
import numpy as np


class MyDisplay(Display):

    def _do(self, problem, evaluator, algorithm):
        super()._do(problem, evaluator, algorithm)
        self.output.append("metric_a", np.mean(algorithm.pop.get("X")))
        self.output.append("metric_b", np.mean(algorithm.pop.get("F")))


problem = get_problem("zdt2")

algorithm = NSGA2(pop_size=100)

res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=1,
               display=MyDisplay(),
               verbose=True)

to add more columns see website

## Interface: Results 

res.X - design space values are

res.F - objective spaces values

res.G - constraint values

res.CV - aggregated constraint violation

res.algorithm - algorithm object

res.pop - final population object

res.history - history of algorithm object. (only if save_history has been enabled during the algorithm initialization)

res.time - the time required to run the algorithm

In [ ]:
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize


problem = get_problem("sphere")
algorithm = GA(pop_size=5)
res = minimize(problem,
               algorithm,
               ('n_gen', 30),
               seed=1)

In [ ]:
res.X, res.F, res.G, res.CV
print(res.X)

In [ ]:
res.algorithm, res.pop

In [ ]:
res.pop.get("X"), res.pop.get("F")

Problem with constraints

In [ ]:
problem = get_problem("g01")
algorithm = GA(pop_size=5)
res = minimize(problem,
               algorithm,
               ('n_gen', 5),
               verbose=True,
               seed=1)

In [ ]:
res.X, res.F, res.G, res.CV

None return means no feasible solution found

Multi-objective example:

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2

problem = get_problem("zdt2")
algorithm = NSGA2()
res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=1)

In [ ]:
res.F

## Problems
n_constr - constraints on objectives

In [ ]:
import numpy as np
from pymoo.model.problem import Problem


class SphereWithConstraint(Problem):

    def __init__(self):
        super().__init__(n_var=10, n_obj=1, n_constr=1, xl=-0, xu=1)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum((x - 0.5) ** 2, axis=1)
        out["G"] = 0.1 - out["F"]

##### Single-Objective Algorithm: Ackley

import autograd.numpy as anp

from pymoo.model.problem import Problem


class Ackley(Problem):

    def __init__(self, n_var=2, a=20, b=1/5, c=2 * anp.pi):
        super().__init__(n_var=n_var, n_obj=1, n_constr=0, xl=-32.768, xu=+32.768, type_var=anp.double)
        self.a = a
        self.b = b
        self.c = c

    def _evaluate(self, x, out, *args, **kwargs):
        part1 = -1. * self.a * anp.exp(-1. * self.b * anp.sqrt((1. / self.n_var) * anp.sum(x * x, axis=1)))
        part2 = -1. * anp.exp((1. / self.n_var) * anp.sum(anp.cos(self.c * x), axis=1))
        out["F"] = part1 + part2 + self.a + anp.exp(1)

    def _calc_pareto_front(self):
        return 0

    def _calc_pareto_set(self):
        return anp.full(self.n_var, 0)

## Algorithms 

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2
algorithm = NSGA2()

from pymoo.factory import get_algorithm
algorithm = get_algorithm("nsga2")

#### NSGA-III

In [ ]:
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_problem, get_reference_directions
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

# create the reference directions to be used for the optimization
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)

# create the algorithm object
algorithm = NSGA3(pop_size=92,
                  ref_dirs=ref_dirs)

# execute the optimization
res = minimize(get_problem("dtlz1"),
               algorithm,
               seed=1,
               termination=('n_gen', 600))

Scatter().add(res.F).show()

In [ ]:
res = minimize(get_problem("dtlz1^-1"),
               algorithm,
               seed=1,
               termination=('n_gen', 600))

Scatter().add(res.F).show()

##### Riesz s-Energy

In [ ]:
from pymoo.factory import get_visualization, get_reference_directions
from pymoo.util.ref_dirs.energy import RieszEnergyReferenceDirectionFactory

ref_dirs = get_reference_directions("energy", 3, 90, seed=1)
#get_visualization("scatter").add(ref_dirs).show()

In [ ]:
ref_dirs = get_reference_directions("energy", 3, 250, seed=1)
#get_visualization("scatter").add(ref_dirs).show()

#### Das-Dennis
n_partitions
\begin{align}
n = C_{p}^{M+p-1} \\
\end{align}
n = number of points

p = number of parameters

M = number of objectives

EXAMPLE:
\begin{align}
p = 10 \\
M = 3 \\
C_{10}^{3+10-1} = \binom{10}{12} = 66 \\
\end{align}

Interior Points
\begin{align}
n_{I} = interior points \\
n_{I} = \binom{p-1}{p-M} \\
\end{align}

In [ ]:
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)
#get_visualization("scatter").add(ref_dirs).show()

In [ ]:
import matplotlib.pyplot as plt

n_dim = range(3,10)
n_points = [len(get_reference_directions("das-dennis", d, n_partitions=12)) for d in n_dim]

plt.plot(n_dim, n_points, '--o')
plt.title("Number of Referenes Directions (p=12)")
plt.ylabel("Number of Points")
plt.xlabel("Dimension")
#plt.show()

#### Multi-layer Approach
bias towards the center

In [ ]:
ref_dirs = get_reference_directions(
    "multi-layer",
    get_reference_directions("das-dennis", 3, n_partitions=12, scaling=1.0),
    get_reference_directions("das-dennis", 3, n_partitions=12, scaling=0.5)
)

#get_visualization("scatter").add(ref_dirs).show()

In [ ]:
ref_dirs = get_reference_directions(
    "multi-layer",
    get_reference_directions("das-dennis", 8, n_partitions=3, scaling=1.0),
    get_reference_directions("das-dennis", 8, n_partitions=2, scaling=0.5),
)

#get_visualization("pcp").add(ref_dirs).show()

No scaling 
Riesz s-Energy method to optimize 
not reference directions directly by scaling

In [ ]:
partitions = [3, 2, 1, 1]

ref_dirs = get_reference_directions("layer-energy", 8, partitions)
#get_visualization("pcp").add(ref_dirs).show()

In [ ]:
from pymoo.util.ref_dirs.energy_layer import LayerwiseRieszEnergyReferenceDirectionFactory

fac = LayerwiseRieszEnergyReferenceDirectionFactory(8, partitions)
ref_dirs = fac.do()

#list(fac.scalings)

In [ ]:
ref_dirs = get_reference_directions("layer-energy", 3, [9, 5, 2, 1])
#get_visualization("scatter").add(ref_dirs).show()

#### Compare NSGA-III to U-NSGA-III

In [ ]:
import numpy as np

from pymoo.algorithms.nsga3 import NSGA3
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("ackley", n_var=30)

# create the reference directions to be used for the optimization - just a single one here
ref_dirs = np.array([[1.0]])

# create the algorithm object
algorithm = UNSGA3(ref_dirs, pop_size=100)

# execute the optimization
res = minimize(problem,
               algorithm,
               termination=('n_gen', 150),
               save_history=True,
               seed=1)

#print("UNSGA3: Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

In [ ]:
_res = minimize(problem,
                NSGA3(ref_dirs, pop_size=100),
                termination=('n_gen', 150),
                save_history=True,
                seed=1)
#print("NSGA3: Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

ret = [np.min(e.pop.get("F")) for e in res.history]
_ret = [np.min(e.pop.get("F")) for e in _res.history]

plt.plot(np.arange(len(ret)), ret, label="unsga3")
plt.plot(np.arange(len(_ret)), _ret, label="nsga3")
plt.title("Convergence")
plt.xlabel("Generation")
plt.ylabel("F")
plt.legend()
#plt.show()

#### R-NSGA-II

In [ ]:
import numpy as np

from pymoo.algorithms.rnsga2 import RNSGA2
from pymoo.factory import get_problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

problem = get_problem("zdt1", n_var=30)
pf = problem.pareto_front()

# Define reference points
ref_points = np.array([[0.5, 0.2], [0.1, 0.6]])

# Get Algorithm
algorithm = RNSGA2(
    ref_points=ref_points,
    pop_size=40,
    epsilon=0.01,
    normalization='front',
    extreme_points_as_reference_points=False,
    weights=np.array([0.5, 0.5]))

res = minimize(problem,
               algorithm,
               save_history=True,
               termination=('n_gen', 250),
               seed=1,
               pf=pf,
               disp=False)


Scatter().add(pf, label="pf").add(res.F, label="F").add(ref_points, label="ref_points").show()

# Paper Problem Example 

In [ ]:
import autograd.numpy as anp
from pymoo.model.problem import Problem
class MyProblem (Problem):
    def __init__ (self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_constr=2,
                         xl=anp.array([-2,-2]),
                         xu=anp.array([2,2]))

    def _evaluate(self,x,out,*args,**kwargs):
        f1 = x[:,0]**2 + x[:,1]**2
        f2 = (x[:,0]*1)**2 + x[:,1]**2
        g1 = 2*( x[:,0]-0.1)*(x[:,0]-0.9) / 0.18
        g2 = -20*(x[:,0]*0.4)*(x[:,0]*0.6) / 4.8
        
        out["F"] = anp.column_stack([f1,f2])
        out["G"] = anp.column_stack([g1,g2])

In [ ]:
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation

algorithm = NSGA2(
    pop_size=40 ,
    n_offsprings=10 ,
    sampling= get_sampling("real_random") ,
    crossover = get_crossover("real_sbx", prob=0.9, 
                              eta=15) ,
    mutation=get_mutation("real_pm", eta=20) ,
    eliminate_duplicates=True
)

In [ ]:
from pymoo.optimize import minimize

res = minimize( MyProblem() ,
               algorithm ,
               ('n_gen', 40),
               seed=1 ,
               verbose=False )

### Gradient

In [ ]:
F, dF = problem.evaluate(np.array([0.1, 0.2]) ,
                         return_values_of =["F", "dF"])
print(F)
print(dF)